In [ ]:
print('hello world')

hello world


In [ ]:
from google.colab import drive
drive.mount('/content/dirve')

Drive already mounted at /content/dirve; to attempt to forcibly remount, call drive.mount("/content/dirve", force_remount=True).


In [ ]:
!pip install torchtext==0.6.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator

import numpy as np

import random
import math
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!pip install janome

  Using cached https://files.pythonhosted.org/packages/a8/63/98858cbead27df7536c7e300c169da0999e9704d02220dc6700b804eeff0/Janome-0.4.1-py2.py3-none-any.whl


In [ ]:
import janome
from janome.tokenizer import Tokenizer
j_t = Tokenizer()

def tokenizer(text):
  return [tok for tok in j_t.tokenize(text, wakati=True)]

In [ ]:
tokenizer("今日は曇りです")

['今日', 'は', '曇り', 'です']

In [ ]:
import torchtext
import torch
from torchtext import data
from torchtext import datasets

In [ ]:
SRC = data.Field(sequential=True, tokenize = tokenizer, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = data.Field(sequential=True, tokenize = tokenizer, init_token='<sos>', eos_token='<eos>', lower=True)

In [ ]:
train, val, test = data.TabularDataset.splits(
        path="/content/dirve/My Drive/Colab Notebooks/", train='train.tsv',
        validation='val.tsv', test='test.tsv', format='tsv',
        fields=[('SRC', SRC), ('TRG', TRG)])

In [ ]:
SRC.build_vocab(train)
TRG.build_vocab(train)

In [ ]:
SRC.vocab.itos

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 'て',
 'に',
 'た',
 'の',
 'が',
 'は',
 'です',
 'を',
 'で',
 'と',
 'こと',
 'し',
 'まし',
 'えー',
 'ね',
 'ん',
 'も',
 'な',
 'ます',
 'ー',
 'から',
 'ない',
 'か',
 'あの',
 'ので',
 'ま',
 'あのー',
 'という',
 'それ',
 'その',
 'おり',
 'お',
 'って',
 'たい',
 '今',
 'いう',
 'よう',
 'い',
 '私',
 '人',
 'けど',
 'っていう',
 'たり',
 'もう',
 'とか',
 'だ',
 'そういう',
 'なっ',
 '一番',
 '思っ',
 'あり',
 '時',
 'やっ',
 'なり',
 'けども',
 'いい',
 'あー',
 'でき',
 '何',
 '自分',
 'まで',
 'う',
 'さん',
 '年',
 'でし',
 'ば',
 'ちょっと',
 'いる',
 'だっ',
 'わたくし',
 '嬉しかっ',
 'たら',
 '行っ',
 'れ',
 '先生',
 '一',
 'する',
 'ませ',
 '思い',
 'けれども',
 'んで',
 '子供',
 'もの',
 'なかっ',
 'ぐらい',
 '遊び',
 '中',
 'えーと',
 'これ',
 '頃',
 '方',
 'あと',
 'くれ',
 'さ',
 'あっ',
 'なる',
 'き',
 'そこ',
 'なんか',
 'たち',
 '仕事',
 'ある',
 '趣味',
 'よ',
 'でも',
 'ところ',
 'なく',
 '好き',
 '来',
 'じゃ',
 'そんな',
 '十',
 'そう',
 'てる',
 '非常',
 'なー',
 '線',
 '2',
 'ござい',
 'また',
 '友達',
 '本当に',
 '嬉しい',
 'この',
 '学校',
 '大学',
 '風',
 'よく',
 '色んな',
 '1',
 'あ',
 'やっぱり',
 '行き',
 '感じ',
 '孫',
 'ながら',
 'それから',
 '勉強',
 '二',
 '

In [ ]:
train_batch_size = 50
test_batch_size = 10
eval_batch_size = 2
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), sort = False,  batch_sizes = (train_batch_size,eval_batch_size, test_batch_size), device= device)

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()
    self.hid_dim = hid_dim
    self.n_layers = n_layers
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
    self.dropout = nn.Dropout(dropout)

  def forward(self, src):
    embedded = self.dropout(self.embedding(src))
    outputs, (hidden, cell) = self.rnn(embedded)
    return hidden, cell

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()
    self.output_dim = output_dim
    self.hid_dim = hid_dim
    self.n_layers = n_layers

    self.embedding = nn.Embedding(output_dim, emb_dim)

    self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

    self.fc_out = nn.Linear(hid_dim, output_dim)
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, input, hidden, cell):
    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input))
    output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
    prediction = self.fc_out(output.squeeze(0))

    return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, src, trg):
    batch_size = trg.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    hidden, cell = self.encoder(src)

    input = trg[0,:]

    for t in range(1, trg_len):
      output, hidden, cell = self.decoder(input, hidden, cell)

      outputs[t] = output
      top1 = output.argmax(1)
    
    return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3652, 32)
    (rnn): LSTM(32, 64, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2349, 32)
    (rnn): LSTM(32, 64, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=64, out_features=2349, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()

  epoch_loss = 0

  for i, batch in enumerate(iterator):

    src = batch.SRC
    trg = batch.TRG
    optimizer.zero_grad()

    output = model(src, trg)

    output_dim = output.shape[-1]
    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    loss = criterion(output, trg)
    loss.backward()
    torch.nn.utils.clip_grad_norm(model.parameters(), clip)
    optimizer.step()

    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
  model.eval()

  epoch_loss = 0

  with torch.no_grad():

    for i, batch in enumerate(iterator):

      src = batch.SRC
      trg = batch.TRG

      output = model(src, trg)

      output_dim = output.shape[-1]

      output = output[1:].view(-1, output_dim)
      trg = trg[1:].view(-1)

      loss = criterion(output, trg)
      epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins*60))
  return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 01 | Time: 0m 48s
	Train Loss: 4.068 | Train PPL:  58.452
	 Val. Loss: 3.625 |  Val. PPL:  37.507
Epoch: 02 | Time: 0m 47s
	Train Loss: 3.665 | Train PPL:  39.059
	 Val. Loss: 3.586 |  Val. PPL:  36.087
Epoch: 03 | Time: 0m 48s
	Train Loss: 3.639 | Train PPL:  38.045
	 Val. Loss: 3.579 |  Val. PPL:  35.832
Epoch: 04 | Time: 0m 48s
	Train Loss: 3.626 | Train PPL:  37.579
	 Val. Loss: 3.609 |  Val. PPL:  36.921
Epoch: 05 | Time: 0m 48s
	Train Loss: 3.617 | Train PPL:  37.229
	 Val. Loss: 3.625 |  Val. PPL:  37.533
Epoch: 06 | Time: 0m 50s
	Train Loss: 3.606 | Train PPL:  36.800
	 Val. Loss: 3.564 |  Val. PPL:  35.306
Epoch: 07 | Time: 0m 47s
	Train Loss: 3.596 | Train PPL:  36.443
	 Val. Loss: 3.542 |  Val. PPL:  34.536
Epoch: 08 | Time: 0m 48s
	Train Loss: 3.590 | Train PPL:  36.235
	 Val. Loss: 3.551 |  Val. PPL:  34.864
Epoch: 09 | Time: 0m 48s
	Train Loss: 3.585 | Train PPL:  36.059
	 Val. Loss: 3.555 |  Val. PPL:  34.981
Epoch: 10 | Time: 0m 48s
	Train Loss: 3.576 | Train PPL

In [ ]:
for i, batch in enumerate(val_iter):
  src = batch.SRC
  trg = batch.TRG

out = model(src, trg)

In [ ]:
def gen_sentence(sentence, src_field, trg_field, model, batch_size):
  model.eval()
  in_str, out_str, pred = [], [], []
  length = len(sentence)

  with torch.no_grad():
    for _, batch in enumerate(sentence):
      src = batch.SRC
      trg = batch.TRG

      output = model(src, trg)
          
      for j in range(min(length, batch_size)):
        topv, topi = output.data.topk(1)
        _, topi_s = output.data.topk(2) 
        for k in range(topi.size()[1]):
          if topi[1:, k][0] == trg_field.vocab.stoi["<eos>"]:
            for i in range(topi_s.size()[0]):
              for l in range(topi_s.size()[1]):
                topi[i][l][0] = topi_s[i][l][1]
          pred.append([trg_field.vocab.itos[topi[1:, k][i]] for i in range(topi.size()[0] - 1)])
        in_str.append([src_field.vocab.itos[i.item()] for i in src[:,j]])
        out_str.append([trg_field.vocab.itos[i.item()] for i in trg[:,j]])
      
  return in_str, out_str, pred

In [ ]:
#中間発表時にはテストデータは用いない
test_in, test_out, test_pred = [],[],[]
test_in, test_out, test_pred = gen_sentence(test_iter, SRC, TRG, model, test_batch_size)
val_in, val_out, val_pred = [],[],[]
val_in, val_out, val_pred = gen_sentence(val_iter, SRC, TRG, model, eval_batch_size)
train_in, train_out, train_pred = [],[],[]
train_in, train_out, train_pred = gen_sentence(train_iter, SRC, TRG, model, train_batch_size)

In [ ]:
import pandas as pd

In [ ]:
def convert_list_to_df(in_list, out_list, pred_list):
  row = []
  for i in range(len(in_list)):
    batch_input = in_list[i]
    batch_output = out_list[i]
    batch_pred = pred_list[i]
    input = [j for j in batch_input if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    output = [j for j in batch_output if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    predict = [j for j in batch_pred if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    input_str = "".join(input)
    output_str ="".join(output)
    predict_str = "".join(predict)
    row.append([input_str, output_str, predict_str])

  df = pd.DataFrame(row, columns=["input","answer","predict"])
  df = df.sort_values('input')
  return df

In [ ]:
train_df = convert_list_to_df(train_in, train_out, train_pred)
val_df = convert_list_to_df(val_in, val_out, val_pred)
test_df = convert_list_to_df(test_in, test_out, test_pred)

In [ ]:
train_df

,input,answer,predict
457,11時半頃,いらっしゃったうーん,あー
7866,11時半頃,十一時半に,あー
2131,15分ぐらいまで,あっ,あですです
6635,15分ぐらいまで,十五分,あ
4986,15分ぐらいまで駅に,あはい,そう
...,...,...,...
9872,麹町番長の大きなお屋敷のおー石造りのバルコニーみたいな部分だけ残っているとか,はー,あです
9082,麻雀をするとか,あ麻雀はい,あんですか
18503,麻雀をするとか,あはい,あ
18881,麻雀をするとか,あーおおおお,あー


In [ ]:
df_s = pd.concat([train_df, test_df]).sort_values('input')

In [ ]:
df_s

,input,answer,predict
457,11時半頃,いらっしゃったうーん,あー
7866,11時半頃,十一時半に,あー
2131,15分ぐらいまで,あっ,あですです
6635,15分ぐらいまで,十五分,あ
4986,15分ぐらいまで駅に,あはい,そう
...,...,...,...
983,麻雀をするとか,あーあはいはい,あーなんですか
9082,麻雀をするとか,あ麻雀はい,あんですか
18503,麻雀をするとか,あはい,あ
6092,麻雀をするとか,あー麻雀も,あはい


In [ ]:
df_s.to_csv("/content/dirve/My Drive/Colab Notebooks/result_seq2seq.csv")